We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `maxh` in `ngmesh`.

In [ ]:
from netgen.csg import *
from ngsolve import *
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions
T_max = 10
current_time = 0
dt = 0.1
h_max = 1

In [ ]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left  = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
    right = Plane(Pnt(1,1,1), Vec( 1,0,0)).bc("right")
    front = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("front")
    back  = Plane(Pnt(1,1,1), Vec(0, 1,0)).bc("back")
    bot   = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("bot")
    top   = Plane(Pnt(1,1,1), Vec(0,0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add (cube)
    #cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry



ngmesh = MakeGeometry().GenerateMesh(maxh=h_max)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [ ]:
  # this helps me figure out which sides are which.
#fes_SCALAR = H1(mesh, order=1)
#myScalars = {"left":10, "right":10, "front":0, "back":0, "bot":5, "top":5}
#SCALAR_gfu = CoefficientFunction([myScalars[val] for val in mesh.GetBoundaries()])

#Draw(SCALAR_gfu, mesh, "SCALAR_gfu")

In [ ]:
fes_mag = VectorH1(mesh, order=1)  # the finite element space for the magnetisation m_h^i
fes_disp = VectorH1(mesh, order=1, dirichlet="bot")  # the finite element space for the displacement u_h^i
print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)

In [ ]:
mag_gfu = Magnetisation_Functions.give_random_magnetisation(mag_gfu)

In [ ]:
print(mag_gfu.vec)
Draw(mag_gfu)